# Langchain Walkthrough

Inspired by: https://langchain-ai.github.io/langgraph/agents/agents/

Setup the environment

In [10]:
import dotenv
import os
import logfire
import json

import tool_functions as tf

import nest_asyncio

nest_asyncio.apply()

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")
logfire_token = os.getenv("LOGFIRE_TOKEN")


# configure logfire
logfire.configure(token=logfire_token)

# 2. Instrumenteer LangChain (automatisch loggen van chains, LLM calls, tools, etc.)
logfire.instrument()

<function logfire._internal.instrument.instrument.<locals>.decorator(func: 'Callable[P, R]') -> 'Callable[P, R]'>

1 Definieer een basis agent

In [11]:
from langgraph.prebuilt import create_react_agent

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")

basic_agent = create_react_agent(
    model=model,
    tools=[],
    prompt=("je bent een agent die op alle vragen een verstandig antwoord geeft"),
)

Stel een vraag aan de agent

In [12]:
# Run the agent
result = basic_agent.invoke(
    {"messages": [{"role": "user", "content": "geef een defintie van een agent"}]}
)

print(result["messages"][-1].content)

Logfire project URL: ]8;id=607572;https://logfire-eu.pydantic.dev/elbrink/claude-test\https://logfire-eu.pydantic.dev/elbrink/claude-test]8;;\

Een agent is een computerprogramma of systeem dat in staat is om zelfstandig beslissingen te nemen en acties uit te voeren om bepaalde doelen te bereiken. Agenten kunnen worden gebruikt in verschillende toepassingen, zoals:

1. Kunstmatige intelligentie: Agenten kunnen worden ingezet om complexe problemen op te lossen, te onderhandelen, te leren en samen te werken met andere agenten.

2. Automatisering: Agenten kunnen taken uitvoeren, processen beheren en beslissingen nemen zonder voortdurende menselijke tussenkomst.

3. Informatiebeheer: Agenten kunnen informatie verzamelen, filteren, organiseren en presenteren op een manier die voor de gebruiker relevant is.

4. Simulatie en modellering: Agenten kunnen worden gebruikt om het gedrag van complexe systemen, zoals ecosystemen of sociale netwerken, te simuleren en te modelleren.

5. Robotica: Agenten kunnen worden gebruikt om autonome robots te besturen en hun gedrag te coördineren.

Essentiële kenmerken van een agent zijn autonomie, soci

Definieer tools voor de agent

In [13]:
tool_agent = create_react_agent(
    model=model,
    tools=[tf.calculate_btw, tf.calculate_discount, tf.get_inwoners_gemeente],
    prompt=("je bent een agent die op alle vragen een verstandig antwoord geeft"),
)

config = {"configurable": {"thread_id": "1"}}

Test een aantal tool calls

In [14]:
result2 = tool_agent.invoke(
    {
        "messages": {
            "role": "user",
            "content": "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld",
        }
    },
    config=config,
)
print(result2["messages"][-1].content)

Je krijgt 15% korting op het bedrag inclusief BTW van 121 euro. Dat betekent een korting van 18,15 euro. De uiteindelijke prijs die je moet betalen is dan 102,85 euro.

Samengevat:
- Oorspronkelijk bedrag excl. BTW: 100 euro
- BTW (21%): 21 euro
- Totaalbedrag incl. BTW: 121 euro 
- Korting (15%): 18,15 euro
- Eindprijs: 102,85 euro


In [15]:
result = tool_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Geef een overzicht van de top-7 gemeenten in Nederland met het grootste aantal inwoners in 2024. Laat ook het aantal inwoners van Groningen zien.",
            }
        ]
    }
)

print(result["messages"][-1].content)

De top-7 gemeenten in Nederland met de grootste bevolking in 2024 zijn:

1. Amsterdam - 931,298 inwoners
2. Rotterdam - 670,610 inwoners 
3. 's-Gravenhage (gemeente) - 566,221 inwoners
4. Utrecht (gemeente) - 374,238 inwoners
5. Eindhoven - 246,417 inwoners
6. Groningen (gemeente) - 243,768 inwoners
7. Tilburg - 229,836 inwoners

Zoals je kunt zien heeft de gemeente Groningen in 2024 243,768 inwoners.


In [16]:
result = tool_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Als alle inwoners uit Eindhoven in 2025 15 euro betalen, hoeveel krijg ik dan.",
            }
        ]
    }
)
print(result["messages"][-1].content)

Dus als alle 249.054 inwoners van Eindhoven in 2025 elk 15 euro betalen, dan komt dat neer op een totaalbedrag van 3.735.810 euro.


Voeg geheugen toe aan de agent

In [17]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent

checkpoint = InMemorySaver()

mem_agent = create_react_agent(
    model=model,
    tools=[tf.calculate_btw, tf.calculate_discount, tf.get_inwoners_gemeente],
    prompt=("je bent een agent die op alle vragen een verstandig antwoord geeft"),
    checkpointer=checkpoint,
)

config = {"configurable": {"thread_id": "1"}}

result = mem_agent.invoke(
    {
        "messages": {
            "role": "user",
            "content": "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld",
        }
    },
    config=config,
)
print(result["messages"][-1].content)

Met 15% korting op het bedrag inclusief BTW van €121 komt de uiteindelijke prijs die je moet betalen uit op €102,85.

Samengevat:
- Oorspronkelijk bedrag excl. BTW: €100
- BTW (21%): €21
- Bedrag incl. BTW: €121
- Korting (15% op €121): €18,15
- Eindprijs: €102,85

Dus je moet uiteindelijk €102,85 betalen voor product X.


In [18]:
result = mem_agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "wat was ook alweer het btw bedrag van het eerder gekochte product X en hoeveel korting kreeg ik. Zoek dit uit de conversatie hiervoor.",
            }
        ]
    },
    config=config,
)
print(result["messages"][-1].content)

De korting was 15% op het bedrag van €121, wat neerkomt op €18,15.
